### 爬取 Ptt 股票版
* Ptt 股票版總共有「標的」 、 「新聞」 、 「心得」 、 「請益」 、 「投顧」 、 「問卷」 、「其他」 、 「公告」 、 「閒聊」等九種類別。本研究選取其中的「心得」 、 「請益」和「閒聊」等三種較有個人主觀情緒的文章作為情緒分析的目標。
* 日期：2015-02-24 至 2025-01-22

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import re

# PTT 股票版首頁
# PTT_URL = "https://www.ptt.cc/bbs/Stock/index.html"
PTT_URL = "https://www.ptt.cc/bbs/Stock/index65.html"
HEADERS = {"User-Agent": "Mozilla/5.0"}

TARGET_CATEGORIES = ["心得", "請益", "閒聊"] # 目標文章類別

START_DATE = "2015-02-24"
END_DATE = "2025-01-22"

posts = []

# 解析完整發文時間
def get_post_date(soup):
    meta_spans = soup.find_all("span", class_="article-meta-value")
    if len(meta_spans) >= 4:
        raw_date = meta_spans[3].text.strip()  # 第 4 個 meta value 通常是日期
        try:
            date_obj = datetime.datetime.strptime(raw_date, "%a %b %d %H:%M:%S %Y")  # 解析格式
            return date_obj.strftime("%Y-%m-%d")
        except ValueError:
            return None
    return None

# 解析文章內容
def get_post_content(soup):
    main_content = soup.select_one("#main-content")
    if not main_content:
        return None
    # 刪除標題、meta 資訊、推文
    for meta in main_content.select(".article-metaline, .article-metaline-right"):
        meta.extract()
    return main_content.text.strip()

# 取得 PTT 文章列表並抓取內容
def get_posts(page_url):
    res = requests.get(page_url, headers=HEADERS)
    if res.status_code != 200:
        if res.status_code == 403 or res.status_code == 503:
            print("❌ 被封鎖了")
        else:
            print("❌ 沒有回覆")
        return None
        
    soup = BeautifulSoup(res.text, "html.parser")
    
    # 解析每篇文章
    for entry in soup.select(".r-ent"):
        title_tag = entry.select_one(".title a")
        if not title_tag:
            continue  # 被刪除的文章

        title = title_tag.text.strip()
        link = "https://www.ptt.cc" + title_tag["href"]

        # 判斷文章類別
        match = re.search(r"\[(.*?)\]", title)
        category = match.group(1) if match else "其他"

        if category in TARGET_CATEGORIES:
            # 爬取貼文內文
            post_res = requests.get(link, headers=HEADERS)
            if post_res.status_code != 200:
                continue
            post_soup = BeautifulSoup(post_res.text, "html.parser")
            
            date = get_post_date(post_soup)
            content = get_post_content(post_soup)

            if date and content:
                posts.append({"date": date, "content": content})
                print("✅ 爬取成功！")

            time.sleep(2)  # 降低請求頻率，避免被封鎖

    # 取得上一頁連結
    prev_link = soup.select_one(".btn-group-paging a:nth-child(2)")
    return "https://www.ptt.cc" + prev_link["href"] if prev_link and "href" in prev_link.attrs else None

# 爬取 PTT 股票版歷史文章
def crawl_ptt_stock():
    page_url = PTT_URL
    while page_url:
        print(f"正在爬取: {page_url}")
        page_url = get_posts(page_url)
        time.sleep(1)
        if len(posts) > 10000:  # 設定合理的最大爬取數量
            break

# 執行爬蟲
crawl_ptt_stock()

# 轉換 DataFrame 並過濾時間範圍
df = pd.DataFrame(posts)
df["date"] = pd.to_datetime(df["date"])
df = df[(df["date"] >= START_DATE) & (df["date"] <= END_DATE)]

# 儲存為 CSV
df.to_csv("./ptt/ptt_stock_posts_08.csv", index=False, encoding="utf-8-sig")

print(f"爬取完成，共 {len(df)} 篇文章，已存入 ptt_stock_posts_08.csv")


正在爬取: https://www.ptt.cc/bbs/Stock/index65.html
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index64.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index63.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index62.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index61.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index60.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index59.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index58.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index57.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index56.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
正在爬取: https://www.ptt.cc/bbs/Stock/index55.html
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
✅ 爬取成功！
